Gentry's Somewhat Homomorphic Cipher Demo
=======================================

This Notebook provides a basic walkthrough of the cryptosystem invented by Gentry [1]. The implementation discussed here is based on the one by Gentry and Halevi [2].

__Note for using Jupyter__: Use $\texttt{SHIFT + ENTER}$ to execute the code snippets!

Recall the __Ideal Coset Problem__:

Given a security parameter $\lambda = 1^n$, a ring $R$ with an ideal $I$, a PPT algorithm $\texttt{Samp}_R$ that samples $R$ according to some distribution, a PPT algorithm $\texttt{GenIdeal}$ and a PPT adversary $\mathcal{A}$, the ICP experiment is as follows:

1. Run $\texttt{GenIdeal}(\lambda, R, I)$ and obtain a tuple $(J^{pk}, J^{sk})$, two generators of the same ideal $J$ such that $I + J = R$.
2. Generate $r_0 \leftarrow \texttt{Samp}_R$ and $r_1 \leftarrow R$ and a bit $b \leftarrow \{0, 1\}$. Then, calculate $r = r_b \mod J^{pk}$.
3. $\mathcal{A}$ is given $R, I, J^{pk}, \texttt{Samp}_R$ and $r$ and outputs some bit $b' \in \{0, 1\}$.
4. Set $\texttt{Expr}^{ICP}_{\mathcal{A}, R, I, \texttt{Samp}_R, \texttt{GenIdeal}}(n) = 1$ if $b = b'$ and 0 otherwise.

Informally, the ICP asks an adversary to decide whether the coset representative it receives was picked uniformly randomly, or whether it was picked according to some different distribution (defined by $\texttt{Samp}_R$). This would be impossible if the custom sampling function was also uniform, but the hardness assumption is that there are non-uniform sampling algorithms for which it is still hard to succeed.

Formally, the __ICP hardness assumption__ says, that the ICP is hard relative to $(R, I ,\texttt{Samp}_R, \texttt{GenIdeal})$ if every PPT adversary $\mathcal{A}$ has negligible probability to succeed, i.e.

$$
\texttt{Adv}^{ICP}_{\mathcal{A}, R, I, \texttt{Samp}_R, \texttt{GenIdeal}}(n) = \left| 2 \cdot \texttt{Expr}^{ICP}_{\mathcal{A}, R, I, \texttt{Samp}_R, \texttt{GenIdeal}}(n) - 1 \right|\leq \text{negl}(n).
$$

The above might seem really far removed, but it has two advantages:
1. It yields a straightforward trapdoor, namely the ideal generators $J^{pk}, J^{sk}$ which will serve as the public and private key.
2. We can build a scheme based on the ICP and then prove its security based on it. Then, we can "substitute in" something more concrete and be guaranteed a secure scheme! (See Section 9.2.3 in the report.)

Now, we pick $R = \mathbb{Z}[x]/f$, where $f(x) = x^N + 1$ for $\lambda = 1^n$ and $N = 2^n$. It can be shown that $f$ is irreducible and hence $R$ is a field. Pick $I = (2) \leq R$. 

Then, define $\texttt{GenIdeal}$ so that it picks some $v \in R$ with its coefficients being between some parameter $t$, such that $v \not\in I$. Then, it calculates some $h \in R$ using $v$ such that it is hard to obtain $v$ from $h$ and $(v) = (h) = J$. Then, $\texttt{GenIdeal}$ returns $J^{pk} = h$ and $J^{sk} = v$. 

Also define $\texttt{Samp}_R$ to pick $u \in R$ such that its coefficients are picked from $\{0, 1\}$, with 0 having some probability $p$ and 1 with probability $1 - p$. This will mean that the norm of $u$ will be extremely small.

In [91]:
load('somewhat_homom_improved.sage')

Key Generation
-------------------------------------------------------------------

Key generation is fairly straight forward based on the particular instance of the ICP decribed above. Given a security parameter $\lambda$, we run $(J^{pk}, J^{sk}) \leftarrow \texttt{GenIdeal}(\lambda)$, and we output $(pk = J^{pk} = h, sk = J^{sk} = v)$. 

However, it is interesting to see how one can obtain the required $h\in J$ (for full details, see Sections 9.3 and 9.4 in the report). 

We are going to take advantage of the fact that $(v)$ is isomorphic to $\mathcal{L}(V) \leq \mathbb{Z}^n$ where $V$ is __rotation basis__ for $v$. Then, we can calculate the "scaled inverse" of $v$ in $R$ by performing the Extended Euclidean Algorithm on $v$ and $f$, obtaining $w \in R$ such that

$$
w \times v = d \mod f, \quad d \in \mathbb{Z}.
$$

Here, we note two things: 

Firstly, directly from the above, we have

$$
    V^{-1} = \frac{1}{d}W.
$$
Secondly, there is a high probability (around a half) that for a randomly picked $v$ there exists an integer $r$, the lattice $\mathcal{L}(V)$ can also be represented by the basis

$$
H = \begin{bmatrix}
        d & 0 & 0 & ... & 0 \\
        r & 1 & 0 & ... & 0 \\
        [r^2]_d & 0 & 1 & ... & 0 \\
        \vdots & \vdots & \vdots & \ddots & \vdots \\
        [r^{n - 1}]_d & 0 & 0 & ... & 1
    \end{bmatrix}
$$

which is the Hermite Normal Form of $V$ and $[\cdot]_d$ is the centered modulo operation by $d$. There are many advantages to finding a $v$ whose HNF is of the above form, one of them being that we can represent the entire public key by just the two integers $d$ and $r$! (See Section 9.4.1 for the details of finding $r$.)

Below, one can check that $pk$ is indeed comprised of only the security parameter $\lambda$ and the numbers $(d, r) = J^{pk}$: 

In [92]:
pk, sk = key_gen(N=7, t = 128)

HNF in correct form after 1 tries.


Encryption
------------------------------------------------------

Given a public key $pk = J^{pk}$, we will encrypt single bit $m \in \{0, 1\}$, by embedding it into an offset from a lattice point. More concretely, we generate $u \leftarrow \texttt{Samp}_R \mod (2)$. Then, we double this polynomial, and embed the message bit into the constant term, by calculating

$$
    a(x) = 2\cdot u(x) + m.
$$

Then, we calculate the coset representative $c(x) = a(x) \mod J^{pk}$ and set it as our ciphertext. Note, that to calculate the above in reality, actually we take advantage of the dual nature of our ideal lattice, and perform the modding in $\mathbb{Z}$. 

It can be shown, that $c$ can be expressed much more compactly, in particular, it can be shown that

$$
    c = [[a(r)]_d, 0, ..., 0]
$$

and therefore the whole ciphertext can just be represented by the first coefficient of $c$!

In [93]:
m1 = 1
m2 = 1
m3 = 0

c1 = encrypt(pk, m1)
c2 = encrypt(pk, m2)
c3 = encrypt(pk, m3)

Decryption
-------------------------------------------------

Given a secret key $J^{sk}$ and a ciphertext (polynomial) $c$, we will calculate

$$
    m' = c \mod J^{sk},
$$
and the message will be the constant term of $m'$, modulo $2$. Again, switching to the lattice context, it can be shown that decryption only requires that it is enough to calculate the following to recover our message:

$$
    [c_1\cdot w_i]_d \mod 2,
$$
where $c_1$ is the constant term of $c$ and $w_i$ is an arbitrary odd coefficient of $w$, the scaled inverse of $v$.

Correctness & Security
-----------------------------------------------------

The correctness and security of the above method is highly non-trivial.
Therefore, these sections are omitted here and we just note that both of them can be proved in terms of the ICP. See Theorem 9.1 for correctness and Theorem 9.2 for the IND-CPA security proof of the abstract scheme in the report.

In [94]:
decrypt(sk, c1)

1

Homomorphic Property of the Scheme
------------------------------------------------

The whole point of the very peculiar and far-removed set-up up until now was the homomorphic property of the scheme. In particular, the decryption function is a limited ring homomorphism between the cipher space and the message space. Limited, because we cannot perform an arbitrary number of operations.

Below, we give a couple of examples of this property.

In [95]:
mod((m1 + m2)^12 + m3 * m1, 2) == decrypt(sk, (c1 + c2)^12 + c3 * c1)

True

In [96]:
decrypt(sk, c1^3 + c3)

1

One might notice, that the above computations may be thought of multivariate polynomials evaluated at the ciphertexts. This is really useful, since boolean circuits correspond 1-to-1 with operations on the message space, hence we can use the equivalent polynomial in the cipherspace to perform the same computation, except on the ciphertexts!

Take for example the full adder circuit:

![Full adder circuit](fulladder.png)

The above performs elementary-school addition: given two numbers and a carry, it adds the three together, giving the sum for the current binary place and calculates the carry for the next binary place.

The above can be turned into polynomials, one for each output. In our scheme, multiplication in $R$ corresponds to $\texttt{AND}$ and addition to $\texttt{XOR}$. So, the above circuit can be rewritten as:

In [97]:
Z.<x, y, z> = ZZ[]
sumC = (x + y) + z
carryC = (x * y) + (x + y) * z

Then, evaluating these at our ciphertexts (and switching to $\mathbb{Z}$ from $\mathbb{Z}[x]$):

In [98]:
sum_ct = ZZ(sumC(x=c1, y=c2, z=c3))
carry_ct = ZZ(carryC(x=c1, y=c2, z=c3))

Finally, decrypting:

In [99]:
decrypt(sk, sum_ct)

0

In [100]:
decrypt(sk, carry_ct)

1

This allows us actually to do quite a lot of interesting things. For example, imagine we wanted to add together the numbers $\texttt{101010}$ and $\texttt{111100}$, we can perform the addition with everything encrypted by building a little ripple-carry adder:

In [101]:
# Use Little-Endian convention
num1 = [0, 1, 0, 1, 0, 1]
num2 = [0, 0, 1, 1, 1, 1]

# Encrypt both numbers
enc_num1 = map(lambda pt: encrypt(pk, pt), num1)
enc_num2 = map(lambda pt: encrypt(pk, pt), num2)

# We start with 0 carry
carry_in = encrypt(pk, 0)

enc_sum = []

for i in range(len(num1)):
    
    # Sum the ith bits
    sum_ct = ZZ(sumC(x=enc_num1[i], y=enc_num2[i], z=carry_in))
    
    enc_sum.append(sum_ct)
    
    # Carry to the i+1th bit
    carry_out = ZZ(carryC(x=enc_num1[i], y=enc_num2[i], z=carry_in))
    
    carry_in = carry_out
    
# Decrypt the result
dec_sum = map(lambda ct: decrypt(sk, ct), enc_sum)

dec_sum

[0, 1, 1, 0, 0, 1]

The above says that $\texttt{101010} + \texttt{111100} = \texttt{100110}$ (mod 64), which in decimal is 42 + 60 = 38 (mod 64), which is correct!

Making the scheme fully homomorphic
--------------------------------------------------------------

Sadly, as discussed above, this scheme is limited to a certain number of operations. For a setting of $N = 128$ and $t = 128$, this limit is around 40 multiplications (the reader might need to play around with the powers, as we can still get an accidental correct decryption):

In [104]:
decrypt(sk, c2^40)

1

This is because our offset vector becomes so big, that it cannot be reliably decrypted. However, the homomorphic property comes to the rescue! In particular, if we can evaluate the decyption function homomorphically, we can "reset" the error vector to a short one. 

The idea is this: given two key pairs $(pk1, sk1), (pk2, sk2)$ and some ciphertext $c$ encrypted under $pk1$, we perform the following:
1. Encrypt $c$ and $sk1$ under $pk2$.
2. Homomorphically decrypt $\texttt{Enc}_{pk2}(c)$ with $\texttt{Enc}_{pk2}(sk1)$

Then, we will be left with some ciphertext encrypted under $pk2$, and our error vector is refreshed. Then, adding on additional key pairs, we may refresh the error as many times as we wish. Finally, in order to avoid calculating newer and newer key pairs, we are just going to use the pair $(pk, \texttt{Enc}_{pk}(sk)$, i.e. in step 1 $c$ is doubly encrypted under the same key, and then we homomorphically decrypt to a singly encrypted state. Thus we achieve full homomorphism!

__Note__: Correctness of the above scheme should be fairly clear. However, it is not trivial that it is secure. In fact, we must modify the scheme somewhat as well as make a few more (mild) assumptions to prove security. However, these considerations are out of the scope of this work.

References
------------------------------------------------

[1] Craig Gentry. A fully homomorphic encryption scheme. Stanford University, 2009.

[2] Craig Gentry and Shai Halevi. “Implementing Gentry’s Fully-Homomorphic Encryption Scheme”.
In: Advances in Cryptology – EUROCRYPT 2011. Ed. by Kenneth G. Paterson. Berlin, Hei-
delberg: Springer Berlin Heidelberg, 2011, pp. 129–148. isbn: 978-3-642-20465-4.